<h2>Multi Layer Perceptron function approximation</h2>

In the following example, we implement an MLP network that will be able to learn the trend of an arbitrary function f(x). In the training phase the network will have to learn from a known set of points, that is x and f(x), while in the test phase the network will deduct the values of f(x) only from the x values.

This very simple network will be built by a single hidden layer. 

Import the necessary libraries:

In [ ]:
import tensorflow as tf
import numpy as np
import math, random
import matplotlib.pyplot as plt

We build the data model. The function to be learned will follow the trend of the cosine function, evaluated for 1000 points to which we add a very little random error (noise) to reproduce a real case:

In [ ]:
NUM_points = 1000
np.random.seed(NUM_points)
function_to_learn = lambda x: np.cos(x) + 0.1*np.random.randn(*x.shape)

Our MLP network will be formed by a hidden layer of 10 neurons:

In [ ]:
layer_1_neurons = 10

The network learns for 100 points at a time to a total of 1500 learning cycles (epochs):

In [ ]:
batch_size = 100
NUM_EPOCHS = 1500

Finally, we construct the training set and the test set:

In [ ]:
#all_x contiene tutti i punti
all_x = np.float32(np.random.uniform(-2*math.pi, 2*math.pi,(1, NUM_points))).T
np.random.shuffle(all_x)
train_size = int(900)

The first 900 points are in the training set:

In [ ]:
x_training = all_x[:train_size]
y_training = function_to_learn(x_training)

The last 100 will be in the validation set:

In [ ]:
x_validation = all_x[train_size:]
y_validation = function_to_learn(x_validation)

Using matplotlib, we display these sets:

In [ ]:
plt.figure(1)
plt.scatter(x_training, y_training, c='blue', label='train')
plt.scatter(x_validation, y_validation,c='red',label='validation')
plt.legend()
plt.show()

<h2>Build the model</h2>
First, we create the placeholders for the input tensor (X) and the output tensor (Y):

In [ ]:
X = tf.placeholder(tf.float32, [None, 1], name="X")
Y = tf.placeholder(tf.float32, [None, 1], name="Y")

Then we build the hidden layer of [1 x 10] dimensions:

In [ ]:
w_h = tf.Variable(tf.random_uniform([1, layer_1_neurons], minval=-1, maxval=1, dtype=tf.float32))
b_h = tf.Variable(tf.zeros([1, layer_1_neurons], dtype=tf.float32))

It receives the input value from the X input tensor, combined with the weight w_hij connections and added with the respective biases of layer 1:

In [ ]:
h = tf.nn.sigmoid(tf.matmul(X, w_h) + b_h)

The output layer is a [10 x 1] tensor:

In [ ]:
w_o = tf.Variable(tf.random_uniform([layer_1_neurons, 1], minval=-1, maxval=1, dtype=tf.float32))
b_o = tf.Variable(tf.zeros([1, 1], dtype=tf.float32))

Each neuron in this second layer receives inputs from the neurons of layer 1, combined with weight w_oij connections and added together with the respective biases of the output layer:

In [ ]:
model = tf.matmul(h, w_o) + b_o

We then define our optimizer for the newly defined model:

In [ ]:
train_op = tf.train.AdamOptimizer().minimize(tf.nn.l2_loss(model - Y))

We also note that in this case, the cost function adopted is the following:

In [ ]:
tf.nn.l2_loss(model - Y)

The tf.nn.l2_loss function is a TensorFlow that computes half the L2 norm of a tensor without the sqrt, that is, the output for the preceding function is as follows:

In [ ]:
output = np.sum((model - Y) ** 2) / 2

<h2>Launch the session</h2>
Let's build the evaluation graph:

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

Now we can launch the learning session:

In [ ]:
errors = []
for i in range(NUM_EPOCHS):
    for start, end in zip(range(0, len(x_training), batch_size), range(batch_size, len(x_training), batch_size)):
        sess.run(train_op, feed_dict={X: x_training[start:end], Y: y_training[start:end]})
    cost = sess.run(tf.nn.l2_loss(model - y_validation), feed_dict={X:x_validation})
    errors.append(cost)
    if i%100 == 0: 
        print("epoch %d, cost = %g" % (i, cost))

The following lines of code allow us to display how the cost changes in the running epochs:

In [ ]:
plt.plot(errors,label='MLP Function Approximation')
plt.xlabel('epochs')
plt.ylabel('cost')
plt.legend()
plt.show()